In [1]:
#//LIBRARIES
import math
import numpy as np
import pandas as pd

import sys
import os 
sys.path.append(os.path.expanduser('~'))

from analysts_tools.growth import *

#Procurement tools
from procurement_lib import send_slack_notification,GoogleSheet,redash
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

from datetime import datetime, timedelta

In [2]:
fecha_inicio = '2024-04-01'

In [3]:
query = """
WITH RECURSIVE calendar(calendar_date) AS (
  SELECT DATE_TRUNC('day', DATE(GETDATE()) - INTERVAL '700 day')
  UNION ALL
  SELECT calendar_date + INTERVAL '1 day'
  FROM calendar
  WHERE calendar_date BETWEEN DATE_TRUNC('day', DATE(GETDATE()) - INTERVAL '700 day') AND DATE(GETDATE() - 1) 
),

prices AS (
WITH info AS (
SELECT 
DATE(p.date_created_at_id) AS fecha,
sup.source_id, 
--sup.description,  
AVG(p.min_gross_price) AS price,
AVG(p.min_pricing_price) AS net_price

FROM dpr_product_pricing.dim_product_price p
INNER JOIN dpr_shared.dim_stock_unit        su  ON su.stock_unit_id = p.stock_unit_id
INNER JOIN dpr_shared.dim_stock_unit        sup ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = sup.category_id AND cat.super_category = 'Multicategoría'

WHERE DATE(date_created_at_id) > DATE(CURRENT_DATE - 700)
 AND p.site_id in (4) --SPO
 AND su.active = 1
 AND su.archived = 0
 AND su.in_catalog = 1
GROUP BY 1,2
),

done AS (
SELECT
  DATE(c.calendar_date) AS quotation_date,
  --s.region,
  --s.parent_product_name,
  (s.source_id)::int as source_id,
  LAG(i.price IGNORE NULLS) OVER (PARTITION BY s.source_id ORDER BY c.calendar_date)::FLOAT AS price,
  LAG(i.net_price IGNORE NULLS) OVER (PARTITION BY s.source_id ORDER BY c.calendar_date)::FLOAT AS net_price
FROM calendar c
CROSS JOIN (SELECT DISTINCT source_id FROM info) s
LEFT JOIN info i ON c.calendar_date = i.fecha-1 AND s.source_id = i.source_id
)

SELECT *
FROM done
WHERE price IS NOT NULL AND net_price IS NOT NULL
 AND DATE(quotation_date) BETWEEN '{fecha}' AND DATE(CURRENT_DATE)
)

SELECT *
FROM prices
""".format(fecha=fecha_inicio)

dfp = run_read_dwd_query(query)
dfp['quotation_date'] = pd.to_datetime(dfp['quotation_date'])

In [4]:
query = """
-- Step 1: Create a helper table for generating date series (if not already created)
CREATE TEMP TABLE date_helper AS
SELECT
    CAST(row_number() OVER () AS INTEGER) - 1 AS n
FROM
    (SELECT 0 AS union_all FROM dpr_shared.dim_date LIMIT 1000) a, -- Adjust limit to ensure enough rows
    (SELECT 0 AS union_all FROM dpr_shared.dim_date LIMIT 1000) b;

WITH
med_bench AS (

-- Step 2: Get the initial data
WITH initial_data AS (
    SELECT
        competitor.competitor_name AS competitor_name,   
        site.identifier_value AS site_code,
        quotation_date.full_date AS quotation_date,
        su.source_id,
        ROUND(cpp.product_selected_price, 2)::float AS price
    FROM dpr_product_pricing.fact_collected_product_prices cpp
    INNER JOIN dpr_shared.dim_date quotation_date ON cpp.dim_quotation_date = quotation_date.date_id
    INNER JOIN dpr_shared.dim_time quotation_time ON cpp.dim_quotation_time = quotation_time.time_id
    INNER JOIN dpr_shared.dim_site site ON cpp.dim_site = site.site_id
    INNER JOIN dpr_shared.dim_category cat ON cpp.dim_category = cat.category_id
    INNER JOIN dpr_product_pricing.dim_product_outlier_type outlier_type ON cpp.dim_outlier_type = outlier_type.outlier_type_id
    INNER JOIN dpr_product_pricing.dim_product_source_type source_type ON cpp.dim_source_type = source_type.source_type_id
    INNER JOIN dpr_product_pricing.dim_product_competitor competitor ON cpp.dim_competitor = competitor.competitor_id
    INNER JOIN dpr_product_pricing.dim_product_competitor_type competitor_type ON (
        CASE
            WHEN cpp.super_category = 'Fruver' THEN competitor.product_competitor_type_id_fruver = competitor_type.competitor_type_id
            WHEN cpp.super_category = 'Multicategoría' THEN competitor.product_competitor_type_id_multicategoria = competitor_type.competitor_type_id
        END
    )
    INNER JOIN dpr_shared.dim_stock_unit su ON cpp.dim_stock_unit = su.stock_unit_id
    WHERE quotation_date.full_date >= DATE('{fecha}') - INTERVAL '9 DAYS'
      AND competitor.competitor_name NOT ILIKE '%cayena%'
      AND site.identifier_value IN ('SPO')
      --AND su.source_id = -314998
      AND competitor.competitor_name <> 'Atacadao_V2'
),
-- Step 3: Create a series of dates using the helper table for each combination of competitor, site_code, and source_id
date_series AS (
    SELECT
        id.competitor_name,
        id.site_code,
        id.source_id,
        dateadd(day, dh.n, id.min_date) AS quotation_date
    FROM
        (SELECT DISTINCT competitor_name, site_code, source_id, MIN(quotation_date) AS min_date, MAX(quotation_date) AS max_date 
         FROM initial_data 
         GROUP BY competitor_name, site_code, source_id) id,
        date_helper dh
    WHERE dateadd(day, dh.n, id.min_date) <= id.max_date + interval '8 day'
),
-- Step 4: Left join the date series with the initial data
joined_data AS (
    SELECT
        ds.competitor_name,
        ds.site_code,
        ds.source_id,
        ds.quotation_date,
        id.price AS original_price
    FROM date_series ds
    LEFT JOIN initial_data id
    ON ds.competitor_name = id.competitor_name
    AND ds.site_code = id.site_code
    AND ds.source_id = id.source_id
    AND ds.quotation_date = id.quotation_date
),
-- Step 5: Fill missing dates with the last known price using window functions
filled_data AS (
    SELECT
        competitor_name,
        site_code,
        quotation_date,
        source_id,
        LAST_VALUE(original_price IGNORE NULLS) OVER (PARTITION BY competitor_name, site_code, source_id ORDER BY quotation_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS filled_price
    FROM joined_data
    WHERE quotation_date BETWEEN (SELECT MIN(quotation_date) FROM initial_data) AND (SELECT MAX(quotation_date) FROM initial_data) + interval '8 day'
)

-- Step 6: Select the required fields
SELECT
    CASE
    WHEN (competitor_name ILIKE '%atacadao%' OR competitor_name ILIKE '%atacadão%') THEN 'MED_atacadao'
    WHEN competitor_name ILIKE '%assaí%' THEN 'MED_assai'
    ELSE 'MED_ALL'
    END AS competitor_name,
    site_code,
    quotation_date,
    source_id,
    MEDIAN(filled_price) AS med_price
FROM filled_data
WHERE filled_price IS NOT NULL
    AND DATE(quotation_date) BETWEEN '{fecha}' AND DATE(CURRENT_DATE)
    --AND (competitor_name ILIKE '%assaí%' OR competitor_name ILIKE 'atacadao%' OR competitor_name ILIKE 'atacadão%')
GROUP BY 1,2,3,4

UNION ALL

SELECT
    'MED_Total_benches' AS competitor_name,
    site_code,
    quotation_date,
    source_id,
    MEDIAN(filled_price) AS med_price
FROM filled_data
WHERE filled_price IS NOT NULL
    AND DATE(quotation_date) BETWEEN '{fecha}' AND DATE(CURRENT_DATE)
GROUP BY 1,2,3,4
)

SELECT *
FROM med_bench
""".format(fecha=fecha_inicio)

dfb = run_read_dwd_query(query)
dfb['quotation_date'] = pd.to_datetime(dfb['quotation_date'])

In [5]:
query = """
SELECT
    s.identifier_value AS region_code,
    DATE(fs.order_submitted_date) AS quotation_date,
    TO_CHAR(DATE(fs.order_submitted_date),'YYYY-MM') AS month,
    TO_CHAR(DATE(fs.order_submitted_date),'YYYY-WW') AS week,
    CASE WHEN cat.parent_description = 'Mercearia' 
         AND cat.description NOT IN ('Arroz', 'Açúcar, adoçantes e doces','Açúcar e adoçantes','Feijão','Grãos','Farinhas e misturas','Azeites, óleos e vinagres')
         THEN 'Despensa'
        ELSE cat.parent_description 
    END AS cat,
    sup.source_id,
    SUM(gmv_pxq)::float AS gmv_usd,
    SUM(gmv_pxq)/SUM(SUM(gmv_pxq)) OVER (PARTITION BY region_code, quotation_date, cat) AS gmv_mix

FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
INNER JOIN dpr_shared.dim_stock_unit        su  ON su.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_stock_unit        sup  ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND cat.super_category = 'Multicategoría'
    AND dp.is_slot = 'false'
    AND fs.gmv_pxq_local > 0
    AND s.identifier_value = 'SPO'
    AND DATE(fs.order_submitted_date) >= '{fecha}'
GROUP BY 1,2,3,4,5,6
""".format(fecha=fecha_inicio)

dfg = run_read_dwd_query(query)
dfg['quotation_date'] = pd.to_datetime(dfg['quotation_date'])

In [6]:
query = """
SELECT 
fecha.full_date AS quotation_date,
sup.source_id,
(CASE WHEN c.dtd_cost_local = 0 THEN m.inventory_p_fin ELSE c.dtd_cost_local END)::float as cost

FROM dpr_cross_business.int_dtd_cost        c  
INNER JOIN dpr_shared.dim_stock_unit        su    ON su.stock_unit_id = c.dim_stock_unit
INNER JOIN dpr_shared.dim_stock_unit        sup   ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id
INNER JOIN dpr_shared.dim_date              fecha ON fecha.date_id = c.dim_date_dtd
LEFT JOIN  dpr_cross_business.fact_cross_business_insights m ON m.dim_stock_unit = su.stock_unit_id AND m.dim_date = c.dim_date_dtd - 1

WHERE fecha.full_date >= '{fecha}'
 AND cost > 0
""".format(fecha=fecha_inicio)

dfc = run_read_dwd_query(query)
dfc['quotation_date'] = pd.to_datetime(dfc['quotation_date'])

In [7]:
df_merge = pd.merge(dfb,dfp, left_on=['source_id','quotation_date'], right_on=['source_id','quotation_date'], how='inner')
df_fb = pd.merge(df_merge,dfc, left_on=['source_id','quotation_date'], right_on=['source_id','quotation_date'], how='inner')

In [8]:
df_fb['gpi'] = df_fb.price/df_fb.med_price
df_fb['npi'] = df_fb.net_price/df_fb.med_price
df_fb['mg'] = 1-(df_fb.cost/df_fb.price)
df_fb['compete'] = 1-(df_fb.cost/df_fb.med_price)

In [9]:
df_final = pd.merge(dfg,df_fb.loc[(df_fb.gpi>0.5) & (df_fb.gpi<2.0)],left_on=['source_id','quotation_date'], right_on=['source_id','quotation_date'], how='inner')

# Logica Mensual

In [10]:
# Assuming df_final is your DataFrame

# Ensure gmv_usd column is numeric for calculations
df_final['gmv_usd'] = pd.to_numeric(df_final['gmv_usd'], errors='coerce')

# First query
grouped1 = df_final.groupby(['region_code', 'month', 'cat', 'competitor_name']).agg({
    'gmv_usd': 'sum',
    'gpi': lambda x: (df_final.loc[x.index, 'gmv_usd'] * x).sum() / df_final.loc[x.index, 'gmv_usd'].sum(),
    'npi': lambda x: (df_final.loc[x.index, 'gmv_usd'] * x).sum() / df_final.loc[x.index, 'gmv_usd'].sum(),
    'mg': lambda x: (df_final.loc[x.index, 'gmv_usd'] * x).sum() / df_final.loc[x.index, 'gmv_usd'].sum(),
    'compete': lambda x: (df_final.loc[x.index, 'gmv_usd'] * x).sum() / df_final.loc[x.index, 'gmv_usd'].sum()
}).reset_index()

# Second query
df_final_multi = df_final.copy()
df_final_multi['cat'] = 'Multi'

grouped2 = df_final_multi.groupby(['region_code', 'month', 'cat', 'competitor_name']).agg({
    'gmv_usd': 'sum',
    'gpi': lambda x: (df_final_multi.loc[x.index, 'gmv_usd'] * x).sum() / df_final_multi.loc[x.index, 'gmv_usd'].sum(),
    'npi': lambda x: (df_final_multi.loc[x.index, 'gmv_usd'] * x).sum() / df_final_multi.loc[x.index, 'gmv_usd'].sum(),
    'mg': lambda x: (df_final_multi.loc[x.index, 'gmv_usd'] * x).sum() / df_final_multi.loc[x.index, 'gmv_usd'].sum(),
    'compete': lambda x: (df_final_multi.loc[x.index, 'gmv_usd'] * x).sum() / df_final_multi.loc[x.index, 'gmv_usd'].sum()
}).reset_index()

# Combine results
final_result_month = pd.concat([grouped1, grouped2], ignore_index=True)

# Logica Semanal

In [11]:
# Assuming df_final is your DataFrame

# Ensure gmv_usd column is numeric for calculations
df_final['gmv_usd'] = pd.to_numeric(df_final['gmv_usd'], errors='coerce')

# First query
grouped1 = df_final.groupby(['region_code', 'week', 'cat', 'competitor_name']).agg({
    'gmv_usd': 'sum',
    'gpi': lambda x: (df_final.loc[x.index, 'gmv_usd'] * x).sum() / df_final.loc[x.index, 'gmv_usd'].sum(),
    'npi': lambda x: (df_final.loc[x.index, 'gmv_usd'] * x).sum() / df_final.loc[x.index, 'gmv_usd'].sum(),
    'mg': lambda x: (df_final.loc[x.index, 'gmv_usd'] * x).sum() / df_final.loc[x.index, 'gmv_usd'].sum(),
    'compete': lambda x: (df_final.loc[x.index, 'gmv_usd'] * x).sum() / df_final.loc[x.index, 'gmv_usd'].sum()
}).reset_index()

# Second query
df_final_multi = df_final.copy()
df_final_multi['cat'] = 'Multi'

grouped2 = df_final_multi.groupby(['region_code', 'week', 'cat', 'competitor_name']).agg({
    'gmv_usd': 'sum',
    'gpi': lambda x: (df_final_multi.loc[x.index, 'gmv_usd'] * x).sum() / df_final_multi.loc[x.index, 'gmv_usd'].sum(),
    'npi': lambda x: (df_final_multi.loc[x.index, 'gmv_usd'] * x).sum() / df_final_multi.loc[x.index, 'gmv_usd'].sum(),
    'mg': lambda x: (df_final_multi.loc[x.index, 'gmv_usd'] * x).sum() / df_final_multi.loc[x.index, 'gmv_usd'].sum(),
    'compete': lambda x: (df_final_multi.loc[x.index, 'gmv_usd'] * x).sum() / df_final_multi.loc[x.index, 'gmv_usd'].sum()
}).reset_index()

# Combine results
final_result_week = pd.concat([grouped1, grouped2], ignore_index=True)

# Imprimir

In [12]:
df_sheet=GoogleSheet("1q5RPXxJ2lyTZNv77SYwow-MDGS-MZc7InblGt7wO6aM")

In [13]:
df_sheet.set_with_dataframe(final_result_month, worksheet='BD_MONTH', clear=True, autocreate=True)
df_sheet.set_with_dataframe(final_result_week, worksheet='BD_WEEK', clear=True, autocreate=True)